# Workflow for simulations of an architectured crop

In [6]:
#| echo: false

# Ignore warning about depreciated modules
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)

In [4]:
# Imports
import numpy as np
import matplotlib.pyplot as plt
from openalea.plantgl.all import Vector3
from oawidgets.plantgl import *

## 1. Parametrized architectural model

ArchiCrop is a Structural Plant Parametric Model (SPPM) which represents the 3D+t visible architecture of a plant shoot (only gramineae for the moment). 

An SPPM is a 3D architectural plant model that can be described at different scales : plant, axis, phytomer, organ. 

A plant is a set of axes. An axis is a finite sequence of phytomers. A phytomer is formed by a pair of an internode and its corresponding leaf.
Internodes and leaves are vegetative organs, as opposed to reproductive organs (e.g. panicles, spikes, pods, etc.).

$$ \text{Plant} = \text{Tree}(\text{organs} \subset \text{phytomers} \subset \text{axes}) $$

The tree structure is organized such that internodes of a given axis are linked to one another, such that the top of internode $I_{i-1}$ corresponds to the bottom of internode $I_i$. Each leaf $L_i$ is attached to the top of the internode $I_i$ of the same rank $i$, forming together the phytomer of rank $i$. For a given phytomer, the growth of a leaf will then start after the beginning of the growth of its corresponding internode.

$$ \text{Phytomer}_i = I_i \cup L_i $$

The architecture of a plant, i.e. its topology, geometry and development, can be defined by parameters at different scales (cf Table) at all thermal time $t \in [0, T_{veg}]$, such that, at the scale of the organ, $\text{Plant}(t) = \bigcup_i (O_i(t))$.

Development parameters are expressed as functions of the phyllochron $\varphi$.

Parameters can be considered as degrees of freedom.



| Symbol   | Description  | Unit    | Scale   | Architecture     |
|----------|--------------|---------|---------|------------------|
| $H$                     | Plant height                         | cm       | Plant              | Geometry       |
| $\varphi$               | Phyllochron                          | $^{\circ}\text{C.day.leaf}^{-1}$  | Plant      | Development   |
| $elong\_internode$      | Elongation rate of internodes        | $^{\circ}\text{C.day.leaf}^{-1}$  | Plant      | Development   |
| $elong\_leaf$           | Elongation rate of leaves            | $^{\circ}\text{C.day.leaf}^{-1}$  | Plant      | Development   |
| $N_{axis}$              | Number of phytomers per axis         | -        | Axis               | Topology       |
| $h_i$                   | Length of an internode $I_i$         | cm       | Organ (internode)  | Geometry       |
| $d_i$                   | Diameter of an internode $I_i$       | cm       | Organ (internode)  | Geometry       |
| $l_i$                   | Length of a leaf $L_i$               | cm       | Organ (leaf)       | Geometry       |
| $w_i$                   | Maximum width of a leaf $L_i$        | cm       | Organ (leaf)       | Geometry       |
| $insertion\_angle_i$    | Insertion angle of a leaf $L_i$ to its axis     | $^{\circ}$         | Organ (leaf)   | Geometry      |
| $scurv_i, curvature_i$  | Leaf curvature parameters for leaf $L_i$        | -                  | Organ (leaf)   | Geometry      |
| $phyllotactic\_angle_i$ | Phyllotactic angle of leaf $L_i$                | $^{\circ}$         | Organ (leaf)   | Geometry      |

*Table: Parameters of ArchiCrop.*

### Organs

#### Internode

Each internode $I_i(t)$ has 3 internal state variables : height (in cm), base and top radii (in cm), and is defined in a local frame. 

First, we assume that the geometry of an internode is represented by a Cylinder. 
We assume a constant radius and its height $h_i$, in $cm$, evolves with $t$ up to a mature height $h_i^{max}$, as described : 

$$[t_i^{start},t_i^{end}] \xrightarrow{~~\chi_i~~} [0,h_i^{max}] \cup [0,d_i^{max}] $$
$$~~~~~~t~~~~~~~~~~ \xrightarrow{~~~~~~~} I_i(\chi_i(t))$$


$$I_i(h_i, d_i) = Cylinder(h_i, d_i)$$



#### Leaf

Each leaf $L_i(t)$ is represented as a curved surface, with an area $s_i$, in $cm^2$, such that its growth up to a mature surface $s_i^{max}$ can be expressed as follows : 

$$[t_i^{start},t_i^{end}] \xrightarrow{~~\psi_i~~} [0,s_i^{max}]$$ + other parameters
$$~~~~~~t~~~~~~~~~~~ \xrightarrow{~~~~~~~} L_i(\psi_i(t))$$

Let's consider leaf surface as a function of leaf length $l$, such that $s = f(l)$. 

- case 1 : Leaves have an homomorphic growth (e.g. for dicots). 

Parameters : width-to-length ratio $r = \frac{w}{l}$, form factor $f$ (Agapie and Sala, 2023) 
    $$s(l_i) = frl_i^2$$
    $$S = frl_{i, max}^2$$
 
- case 2 : Leaves grow along the midrib (e.g. for monocots). 

Parameters : curve of the evolution of width along midrib 
    $$s(l_i) = 2 \times \int\limits_0^{l_i} C(u) du$$
    $$S = 2 \times \int\limits_0^{l_{i, max}} C(u) du$$



### Branching

| Symbol | Description | Unit | Scale | Architecture |
|--------------------------------------|---------------------------------------------------------------------------------------|--------------------------|---------------------------|--------------------------|
| $B$ | Number of branches in the plant | - | Plant | Topology |
| $B_{axis}$ | Number of branches per axis | - | Axis | Topology |
| $order_{axis}$ | Order of ramification | - | Axis | Topology |
| $ramif\_rate_{axis}$ | Ramification rate | $^{\circ}\text{C.day.leaf}^{-1}$ | Axis  | Development  |
| $insertion\_angle_{axis}$  | Insertion angle of an branched axis to its bearing axis  | $^{\circ}$   | Axis  | Geometry  |
| $curvature_{axis}$  | Axis curvature  | -   | Axis  | Geometry  |

*Table: Additional parameters for a ramified plant.*

### Senescence

| Symbol | Description | Unit | Scale | Architecture |
|--------------------------------------|---------------------------------------------------------------------------------------|--------------------------|---------------------------|--------------------------|
| $senescence$ | Leaf lifespan before senescence | $^{\circ}\text{C.day.leaf}^{-1}$ | Plant | Development |

*Table: Additional parameters for senescence.*

### Implementation

We use a Multiscale Tree Graph (MTG) to represent the architecture of a plant at different scales, i.e. a plant, an axis, a phytomer and an organ can be considered as vertices given the scale considered. In parallel, edges connect axes, phytomers and organs in tree structures at different scales.

ArchiCrop is inspired by ADEL (Fournier et al., 2003). It inherits from its modules managing the generation of plant architecture (topology, geometry, development) for given parameters. 
The difference of ArchiCrop mainly lies in the rules for plant growth and the computation of organ features (e.g. internode length, leaf length, etc.).
Growth (and later senescence) of a plant is constrained from temporal series (height(t), leaf area(t), etc) obtained from simulation or measure (cf 3.).

## 2. Allometric laws on architectural parameters 

Allometric laws enable to :
- generate more realistic plants corresponding to given species;
- restrain the space of parameters;
- reduce the number of degrees of freedom in the parametric model.

| Function                          | Description                                                                    | Parameters |
|-----------------------------------|--------------------------------------------------------------------------------|------------|
| $d_{i}=f(age)$                    | Evolution of internode diameter as a function of age ? rank ?                  | |     
|                                   | Geometric law for the distribution of internode lengths along the axis         | $stem_q$ | 
|                                   | Weibull law for the distribution of mature leaf lengths along an axis          | $rmax, skew, l_{i,max}^{max}$ | 
| $w = p(l)$                        | Evolution of width along midrib                                                | $\alpha, r=\frac{w}{l}$ | 
| $reduc\_ramif(order_{axis})$      | Reduction factor for ramified axes                                             | $reduc\_ramif$ | 

*Table: Allometric laws of ArchiCrop.*

Graph for each law

## 3. Organs growth

We assume that it is possible to constrain the growth of a plant generated with ArchiCrop with time series. These time series can be either measurements or the outputs of a crop model simulation. 

The growth of architectured plant organs does not follow explicit ecophysiological laws, and is not directly conditioned by environmental resources. Indeed it is calculated using an integro-differential approach.

As plant architecture is explicitly represented in ArchiCrop, the constraint can be distributed over the different elements of the architecture in different ways. 

Architectural parameters constitute degrees of freedom, not considered at the scale of the crop model. For a simulation with a crop model, it is therefore possible to obtain a large number of simulations on a finer scale, by varying these degrees of freedom within a given range of values for a given species. In other words, we can explore the architectural variability of a species by generating a set of morphotypes.

### 3.1. Compute start and end of organ growth based of phyllochron and elongation rate

In [5]:
from openalea.archicrop.dynamic import thermal_time

### 3.2. Retrieve growth increments for given thermal time steps

The growth of the vegetative part of a plant can be considered as the dynamics of its height $H(t)$ and leaf area $S(t)$ over thermal time $t$. From known dynamics at the scale of the plant, we assume that we can constrain the growth of each internode $I_i$ and leaf $L_i$, by distributing individually the gain of, respectively, height $dh$ and leaf area $dS$ at each time step $dt$.

Given a simulation with a crop model (or any source of constraint at the crop scale) :

| Symbol                 | Description                                                                 | Unit                |
|------------------------|-----------------------------------------------------------------------------|---------------------|
| $t \in [0, T_{veg}]$    | Thermal time series to the end of the vegetative phase $T_{veg}$ (might be corrected with stresses) | $^{\circ}\text{C.day}$ |
| $H(t)$                 | Plant height dynamics                                                       | cm                  |
| $S(t)$                 | Plant leaf area dynamics                                                     | cm                  |

*Table: Inputs to give to ArchiCrop.*

In [2]:
def read_columns_from_file(filename, separator=';'):
    '''Read a file column-wise and store the values in a 2D-list.'''
    with open(filename, 'r') as file:
        columns = []
        for line in file:
            values = line.strip().split(separator) # Strip any extra whitespace and split the line by the separator
            if not columns:
                columns = [[] for _ in values]
            while len(columns) < len(values):
                columns.append([])
            for i, value in enumerate(values):
                columns[i].append(value)
            # Handle lines with fewer values than columns
            for i in range(len(values), len(columns)):
                columns[i].append('')
    return columns

In [3]:
## e.g. from a STICS simulation for maize

filename = 'mod_smaize.sti'
columns = read_columns_from_file(filename)
columns = columns[5:]

# Cut the simulation at the end of vegetative growth
end = 115 
# To get an equal LA/plant 
density = 0.00095 # density = 9.5 plant/m2 = 0.00095 plant/cm2
la = np.array([float(i)/density for i in columns[0][18:end]]) # leaf area
height = np.array([float(i) for i in columns[1][18:end]]) # height
tt = np.cumsum([float(i) for i in columns[2][18:end]]) # thermal times

### Compute whole plant architectures for all time steps


$$\frac{dPlant(t)}{dt} = ArchiCrop(\frac{dLAI}{dt}, \frac{dHeight}{dt})$$

#### Distribute growth among plants


#### Distribute growth among organs 

H0: All growing vegetative organs of a given type grow at the same speed on a given time slot, i.e. equal distribution of the gain among all growing vegetative organs. 

For a growing vegetative organ $i$ over $n$ growing vegetative organs in a plant, given a constraint $dc(t)$ for the plant a time $t$ : 
$$dc_{i}(t) = \frac{dc(t)}{n}$$

H1: Growing vegetative organs receive a gain proportional to their sink strength related to their age until end of growth. 

For a growing vegetative organ $i$ over $n$ growing vegetative organs in a plant (set $G^n$), of ages $a_i$, sink strength beta function $\beta (a_i)$ and relative sink strength $p_i(a_i)$, given a constraint $dc(t)$ for the plant a time $t$ : 
$$p_i(a_i) = \frac{\beta (a_i)}{\sum_{j \in G^n} \beta (a_j)}$$
$$dc_i(t) = p_i(a_i) dc(t)$$

______________________________________

A growing plant is composed of mature phytomers and growing phytomers, such that, for all $t$ in $[0,T_{veg}]$ : 
$$Plant(t) = (\cup phytomers_{mature} (t \geq t^{end})) \cup (\cup phytomers_{growing} (t^{start} < t < t^{end}))$$
such that : 
$$phytomers(t) \subset phytomers$$



- case 1 : All phytomers grow sequentially, i.e. one after the other.
    For all $i$ in $[0, N]$ :
    $$t^{start}_i = i \times \varphi = t^{start}_{i-1, '<'} + \varphi$$
    $$t^{end}_i = t^{start}_{i} + ligul = i \times \varphi + ligul $$

    where $\varphi$ = phyllochron = constant, 
    and $ligul$ = ligulochron = $\alpha \varphi$.

    For ramifications, which can develop with a delay, i.e. if edge(parent(i),i) = '+' :
    $$t^{start}_i = t^{start}_{i-1, '+'} + \varphi + delay$$

    where $delay = \beta \varphi$. 

    Let's assume that all phytomers have the same constant growth rate, i.e. $\chi$ and $\psi$ are linear functions. 
    
    For all $t$ in $[0,T_{veg}]$, for all $i$ in $[0, N]$, we want to find $\frac{dh_i(t)}{dt}$ for all $I_i$ and $\frac{ds_i(l_i(t))}{dt}$ for all $L_i$. 

    At the scale of the plant, at each time step $dt$, we can obtain from a crop model : 


    $$\frac{dPlant(t)}{dt} = \frac{dI(t)}{dt} \cup \frac{dL(t)}{dt}$$

    Then, for all $phytomers_{growing} (t^{start} < t < t^{end})$, 
    $$\frac{dh_i(t)}{dt} = \frac{1}{N_{growing}} \frac{dh(t)}{dt}$$
    $$\frac{ds_i(l_i(t))}{dt} = \frac{1}{N_{growing}} \frac{ds(t)}{dt}$$
    For leaves, $\frac{l_i(t)}{dt}$ can then be computed depending on the case (cf 3.3).

    
  - case 1 : Leaves have an homomorphic growth (e.g. for dicots). 
        $$\frac{l_i(t)}{dt} = \sqrt{\frac{1}{r} \frac{ds_i(l_i(t))}{dt}}$$
    
  - case 2 : Leaves grow along the midrib (e.g. for monocots). 
        Analytic resolution if the function $f$ of the curve $C$ is known and is $C^1$ on $[0,l_{max}]$ : 
        $$F(l_i(t)) = \frac{1}{2} \frac{ds_i(l_i(t))}{dt} - F(l_{i-1}(t))$$
        where $F$ is the antiderivative of $f$. 
        Numeric resolution if too complicated. 




#### Constrain internode growth

We are given at each time step $t$ a value for the plant height, $H(t)$, which corresponds to the sum of the length of the internodes, mature and developing, of the main axis at time $t$, such that :

$$
    H(t) = \sum_{i_{\text{main axis}}} h_i(t)
$$

Starting from a C1 curve of plant height dynamics, $H = f(t)$, it is possible to derive a height increment $dH(t)$ for all $dt$. How to distribute this increment among $n$ growing internodes ? Each internode $I_i$ has a height $h_i(t)$ for a given thermal time $t$. 

For a plant composed of a single axis with a single internode, we obtain, for all $t \in [0, T_{veg}]$ : 

$$h_{single}(t+dt) = h_{single}(t) + dH(t)$$

For a plant composed of a single axis bearing two internodes, we define a phyllochron $\varphi$, in $^{\circ}\text{C.day.leaf}^{-1}$, that is the rate of apparition of successive internodes. 

H0 : Internodes grow sequentially, which means that they appear at the end of the growth of the previous one. 

For all $t \in [0, T_{veg}]$ :

$$h_1(t+dt) = h_1(t) + dH(t), \text{ if } t  < \varphi $$
$$h_2(t+dt) = h_2(t) + dH(t), \text{ if } t  \geq \varphi $$

H1 : Internodes can grow simultaneously, which means that at least two internodes can be growing at the same time step, AND all internodes have the same growth rate. 

Let's define a ligulochron $ligul$ as the duration of internode growth, as a function of the phyllochron, such that $ligul = \alpha \varphi$.

For all $t \in [0, T_{veg}]$ :

$$h_1(t+dt) = h_1(t) + dH(t), \text{ if } t  < \varphi $$

$$h_1(t+dt) = h_1(t) + \frac{1}{2} dH(t), \text{ if } t  \geq \varphi \cup t < ligul $$
$$h_2(t+dt) = h_2(t) + \frac{1}{2} dH(t), \text{ if } t  \geq \varphi \cup t < ligul $$

$$h_2(t+dt) = h_2(t) + dH(t), \text{ if } t  \geq \varphi \cup t \geq ligul $$



#### Constrain leaf growth

We are given at each time step $t$ a value for the leaf area of a plant, $S(t)$, which corresponds to the sum of the surfaces of all the leaves, mature and developing, of the plant at time $t$, such that :

$$
    H(t) = \sum_i s_i(t)
$$

Starting from a C1 curve of plant leaf area dynamics, $S = f(t)$, it is possible to derive a surface increment $dS(t)$ for all $dt$. How to distribute this increment among $n$ growing leaves ? Each leaf $L_i$ has a height $s_i(t)$ for a given thermal time $t$. 

For a plant composed of a single leaf, we obtain, for all $t \in [0, T_{veg}]$ : 

$$h_{single}(t+dt) = h_{single}(t) + dH(t)$$



For a plant composed of two leaves, we define a phyllochron $\varphi$, in $^{\circ}\text{C.day.leaf}^{-1}$, that is the rate of apparition of successive internodes. 



### Plants as part of a crop (planter)

### Compute process (light interception) for various parameters

Caribu

Varying parameters

at all thermal time $t$

### Compare with crop model (Beer's law for light interception)